In [1]:
import json
import pandas as pd
import unicodedata

def clean_unicode(text):
    return unicodedata.normalize("NFKC", text)

In [5]:
infer_data = pd.read_json('./data/loco_test_conspemollm.json', lines=True)
instruction_list = infer_data.apply(
    lambda row: pd.Series(
        {
            "instruction": clean_unicode(
                f"Human: \n" + row["instruction"] + "\n\nAssistant:\n"
            )
        }
    ),
    axis=1,
)["instruction"].to_list()
print(instruction_list[0])

Human: 
Task: Determine if the text is a conspiracy theory. Classify it into one of the following two classes: 0. non-conspiracy. 1. conspiracy.
Text: Coronavirus is indiscriminate. We have seen young and old affected, rich and poor. In the UK alone, we have seen the Prime Minister himself succumb to it and even Prince Charles. But this pandemic has repercussions beyond our nation states. In a short time, it has paralysed the ‘developed’ world and threatened its very existence.

The unipolar world as we know it is being questioned like never before. Nations demonised in the West – Russia, China and Cuba – have been the very countries sending aid to the West.

Never has one witnessed a more ironic act than the US accepting aid from Russia, its arch geopolitical enemy and ‘the primary threat to American interests’ – according to a survey published last year by a US think-tank. The Antonov An-124-100 cargo plane delivered vital medical equipment and masks from Moscow to the US on Wednesda

In [14]:
# get first row
if 'output' in infer_data.iloc[0]:
    print('yes')

yes


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

max_new_tokens = 1024
generation_config = dict(
    temperature=0.9,
    top_k=30,
    top_p=0.6,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=max_new_tokens,
)

load_type = torch.float16  # Sometimes may need torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('./model/ConspEmoLLM-7b')
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"
model_config = AutoConfig.from_pretrained('./model/Emollama-chat-7b')
model = AutoModelForCausalLM.from_pretrained(
    './model/Emollama-chat-7b',
    torch_dtype=load_type,
    config=model_config,
    device_map="auto",
)

if device.type == "cpu":
    model.float()

model.eval()
print("Load model successfully")
print(model.config.max_position_embeddings)

In [4]:
from huggingface_hub import snapshot_download

model_path = snapshot_download(repo_id="lzw1008/ConspEmoLLM-7b", local_dir="./model/ConspEmoLLM-7b")

print("Model downloaded to:", model_path)

Fetching 12 files: 100%|██████████| 12/12 [02:56<00:00, 14.70s/it]

Model downloaded to: /Users/honhaochen/Documents/ConspEmoLLM/model/ConspEmoLLM-7b


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('./model/ConspEmoLLM-7b')
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"

input_txt = "testing, hello world"
truncated_text = tokenizer.decode(
    tokenizer(input_txt, truncation=True, max_length=1800, add_special_tokens=False)["input_ids"],
    skip_special_tokens=True,
    spaces_between_special_tokens=False,
)
print("Truncated text:", truncated_text)

Truncated text: testing, hello world


both ConspEmoLLM and Emollama-chat-7b has max_position_embeddings of 2048
